# Compare Volatility Between PR Models

In [1]:
import sys
sys.path.append('../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

from pyomo.network import Arc, SequentialDecomposition

from idaes.core.util.initialization import propagate_state

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

import idaes.logger as idaeslog
from pyomo.opt import TerminationCondition, SolverStatus

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Import unit models from the model library
from idaes.generic_models.unit_models import Flash

# Import R32, R125, IL info file
from hfc32_emimtf2n_PR import configuration

In [2]:
m1 = ConcreteModel()

m1.fs = FlowsheetBlock(default={"dynamic": False})

m1.fs.properties = GenericParameterBlock(default=configuration)

m1.fs.F101 = Flash(default={"property_package": m1.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m2 = ConcreteModel()

m2.fs = FlowsheetBlock(default={"dynamic": False})

m2.fs.properties = GenericParameterBlock(default=configuration)

m2.fs.F101 = Flash(default={"property_package": m2.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m3 = ConcreteModel()

m3.fs = FlowsheetBlock(default={"dynamic": False})

m3.fs.properties = GenericParameterBlock(default=configuration)

m3.fs.F101 = Flash(default={"property_package": m3.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

    (ordered) position is deprecated.  Please use at()  (deprecated in 6.1,
    will be removed in 7.0) (called from /Users/bridgettebefort/Dowling-
    Maginn-Lab/DowlingLab/IL-MD/idaes-
    pse/idaes/generic_models/properties/core/state_definitions/FTPx.py:178)


In [3]:
m1.fs.properties.PR_kappa_A['R32', 'emimTf2N'].fix(-0.020541972796452503)
m1.fs.properties.PR_kappa_A['emimTf2N', 'R32'].fix(-0.02894149152636121)

m2.fs.properties.PR_kappa_A['R32', 'emimTf2N'].fix(0.09599152317130334)
m2.fs.properties.PR_kappa_A['emimTf2N', 'R32'].fix(1.0036378035313152)
m2.fs.properties.PR_kappa_B['R32', 'emimTf2N'].fix(-0.14148134212147054)
m2.fs.properties.PR_kappa_B['emimTf2N', 'R32'].fix(-1.163092210073511)

m3.fs.properties.PR_kappa_A['R32', 'emimTf2N'].fix(0.4969906835938559)
m3.fs.properties.PR_kappa_A['emimTf2N', 'R32'].fix(4.2604200711470135)
m3.fs.properties.PR_kappa_B['R32', 'emimTf2N'].fix(-0.8976688948244601)
m3.fs.properties.PR_kappa_B['emimTf2N', 'R32'].fix(-7.359503058312826)
m3.fs.properties.PR_kappa_C['R32', 'emimTf2N'].fix(2.9287932976059152)
m3.fs.properties.PR_kappa_C['emimTf2N', 'R32'].fix(0.35439612180100655)

In [4]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [5]:
# Set pressures to loop over
R32_z = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

# Make outlet arrays
IL_x1 = np.zeros(len(R32_z))
IL_y1 = np.zeros(len(R32_z))
R32_x1 = np.zeros(len(R32_z))
R32_y1 = np.zeros(len(R32_z))

IL_x2 = np.zeros(len(R32_z))
IL_y2 = np.zeros(len(R32_z))
R32_x2 = np.zeros(len(R32_z))
R32_y2 = np.zeros(len(R32_z))

IL_x3 = np.zeros(len(R32_z))
IL_y3 = np.zeros(len(R32_z))
R32_x3 = np.zeros(len(R32_z))
R32_y3 = np.zeros(len(R32_z))

In [6]:
m1.fs.F101.inlet.flow_mol.fix(10)
m1.fs.F101.inlet.temperature.fix(298)
m1.fs.F101.inlet.mole_frac_comp[0, "emimTf2N"].fix(0.5) 
m1.fs.F101.inlet.pressure.fix(100000)

m2.fs.F101.inlet.flow_mol.fix(10)
m2.fs.F101.inlet.temperature.fix(298)
m2.fs.F101.inlet.mole_frac_comp[0, "emimTf2N"].fix(0.9) 
m2.fs.F101.inlet.pressure.fix(100000)

m3.fs.F101.inlet.flow_mol.fix(10)
m3.fs.F101.inlet.temperature.fix(298)
m3.fs.F101.inlet.mole_frac_comp[0, "emimTf2N"].fix(0.9) 
m3.fs.F101.inlet.pressure.fix(100000)

In [7]:
m1.fs.F101.vap_outlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.temperature.fix(298.15)
m2.fs.F101.vap_outlet.pressure.fix(100000)
m2.fs.F101.vap_outlet.temperature.fix(298.15)
m3.fs.F101.vap_outlet.pressure.fix(100000)
m3.fs.F101.vap_outlet.temperature.fix(298.15)

In [8]:
m1.fs.liq = Param(mutable=True, default=0.3)
m2.fs.liq = Param(mutable=True, default=0.3)
m3.fs.liq = Param(mutable=True, default=0.3)

In [9]:
for i in range(len(R32_z)):
    print("Trying",R32_z[i])
    
    m1.fs.F101.inlet.flow_mol.fix(10)
    m1.fs.F101.inlet.temperature.fix(298)
    m1.fs.F101.inlet.mole_frac_comp[0, "emimTf2N"].fix(0.7) 
    m1.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.3*R32_z[i])
    m1.fs.F101.inlet.pressure.fix(100000)

    m2.fs.F101.inlet.flow_mol.fix(10)
    m2.fs.F101.inlet.temperature.fix(298)
    m2.fs.F101.inlet.mole_frac_comp[0, "emimTf2N"].fix(0.7) 
    m2.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.3*R32_z[i])
    m2.fs.F101.inlet.pressure.fix(100000)

    m3.fs.F101.inlet.flow_mol.fix(10)
    m3.fs.F101.inlet.temperature.fix(298)
    m3.fs.F101.inlet.mole_frac_comp[0, "emimTf2N"].fix(0.7) 
    m3.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.3*R32_z[i])
    m3.fs.F101.inlet.pressure.fix(100000)

    m1.fs.F101.vap_outlet.pressure.fix(100000)
    m1.fs.F101.vap_outlet.temperature.fix(298.15)
    m2.fs.F101.vap_outlet.pressure.fix(100000)
    m2.fs.F101.vap_outlet.temperature.fix(298.15)
    m3.fs.F101.vap_outlet.pressure.fix(100000)
    m3.fs.F101.vap_outlet.temperature.fix(298.15)   
    
    m1.fs.liq = R32_z[i]
    m2.fs.liq = R32_z[i]
    m3.fs.liq = R32_z[i]

    m1.fs.F101.initialize(outlvl=idaeslog.CRITICAL)              
    m2.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
    m3.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
    
    # solve the model
    status1 = solver.solve(m1, tee = False)
    status2 = solver.solve(m2, tee = False)
    status3 = solver.solve(m3, tee = False)
        
    # If solution is optimal store the concentration, and calculated temperatures in the created arrays
    if (status1.solver.status == SolverStatus.ok) and (status1.solver.termination_condition == TerminationCondition.optimal):

        IL_x1[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"])
        IL_y1[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "emimTf2N"])
        R32_x1[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
        R32_y1[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
        
        print("Model 1")

    if (status2.solver.status == SolverStatus.ok) and (status2.solver.termination_condition == TerminationCondition.optimal):
        IL_x2[i] = value(m2.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"])
        IL_y2[i] = value(m2.fs.F101.vap_outlet.mole_frac_comp[0, "emimTf2N"])
        R32_x2[i] = value(m2.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
        R32_y2[i] = value(m2.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
        
        print("Model 2")

    if (status3.solver.status == SolverStatus.ok) and (status3.solver.termination_condition == TerminationCondition.optimal):  

        IL_x3[i] = value(m3.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"])
        IL_y3[i] = value(m3.fs.F101.vap_outlet.mole_frac_comp[0, "emimTf2N"])
        R32_x3[i] = value(m3.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
        R32_y3[i] = value(m3.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
        
        print("Model 3")

        #Change this to feasible
        print("Feasible")

    else:
        print('Infeasible.')

Trying 0.1
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
Model 1
Model 3
Feasible
Trying 0.2
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
Model 1
Model 3
Feasible
Trying 0.3
Model 1
Model 2
Model 3
Feasible
Trying 0.4
Model 1
Model 2
Model 3
Feasible
Trying 0.5
Model 1
Model 2
Model 3
Feasible
Trying 0.6
Model 1
Model 2
Model 3
Feasible


In [ ]:
alpha1 = np.zeros(len(R32_z))
alpha2 = np.zeros(len(R32_z))
alpha3 = np.zeros(len(R32_z))

for i in range(len(R32_z)):
    alpha1[i] = (R32_y[i]/R32_x[i])/(R32_y[i]/R32_x[i])
    alpha2[i] = (R125_y2[i]/R125_x2[i])/(R32_y2[i]/R32_x2[i])
    alpha3[i] = (R125_y3[i]/R125_x3[i])/(R32_y3[i]/R32_x3[i])


fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(1, 1, 1)

plt.title('Volatility',fontsize=20)
plt.ylabel(r'$\alpha_{12}$',fontsize=20)
plt.xlabel('HFC-125 Mole Fraction',fontsize=20)
plt.grid(True)
plt.plot(R125_x/0.02687632,alpha1,"b",label="Ternary")
plt.plot(R125_x2/0.02625086,alpha2,"g",label="Binary")
plt.plot(R125_x3/0.02625086,alpha3,"r",label="Binary T dep")
plt.ylim(0,3.8)
plt.xlim(0.1,1)
plt.legend()

plt.show()